In [1]:
import pandas as pd
import numpy as np
import spacy 
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)
import re
from geopy import geocoders  
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
from opencage.geocoder import OpenCageGeocode
import folium
tweets_df = pd.read_csv('/Tweets10SEP2019.csv')
states_df = pd.read_csv('uscities.csv', error_bad_lines=False)

In [2]:
listgeotweets = list(tweets_df['user.location'].to_dict().values())
geotw = [x for x in listgeotweets if str(x) != 'NaN']
geotw = [x for x in listgeotweets if str(x) != 'nan']
geotw = [i for i in geotw if not i.isalpha()]               #list of USER.LOCATION VALUES SOME ARE NOT ACUTAL LOCATIONS
locationsDB = list(states_df['citypos'].to_dict().values()) #list of officialcities 
strlocations = ', '.join(locationsDB)
citycoordsdict = dict(zip(states_df.citypos, states_df.LATLONG)) #makes a CITY,STATE and LATLONG Dictionary FROM OFFICAL DB
textandlocation = dict(zip(tweets_df['user.location'], tweets_df['text']))

In [3]:
#USING SPACY, WE COMPARE geotw TO  citycoordsdict to find values that are actual locations:
strgeo = ' ,'.join(geotw) #turn geotw list (values thhat may or may not be cities) into string
doc = nlp(strgeo) #tokenize every word in tthe string of possible cities
phrase_patterns = [nlp(text) for text in locationsDB] #tokenizes every word in list of official us CITIES
matcher.add('Country',None,*phrase_patterns)
found_matches = matcher(doc) #looks for matches 
nord_tweetlocations = [] # empty list that will be the list off official cities from USER.LOCATION
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id] #get string representation
    span = doc[start:end]                   #get the matched span
    nord_tweetlocations.append(span.text) #add to list
    #print(match_id, string_id, start, end, span.text) #uses spacy to get locations from user.locations

In [4]:
#nordtweets is a list of of ACTUAL USERLOCATIONS FROM USER TWEETS
# using list comprehension 
# getting selective dictionary keys 
# compares nord_tweetlocations to dictionary of ocficial cities and lat and longs , returns lat and longs
nordlatlong = [citycoordsdict[i] for i in nord_tweetlocations if i in citycoordsdict]
nordtextcoord = [textandlocation[i] for i in nord_tweetlocations if i in textandlocation]

In [5]:
#now we have two list, nordtweetsCITIESSTATES and nordtweetsLATLONG, we put them into a dictionary
nordgeodict = {'cities':nord_tweetlocations,'coords':nordlatlong}#'tweet': nordtextcoord
customerlocations = pd.DataFrame(nordgeodict) #dataframe of OFFICIAL USER LOCATIONS 

In [6]:
#now we insert a COLUMN called 'count' where each value will have 1, this if for the heat map
#customerlocations.insert(0, 'count', 1)

In [7]:
lat = []
lon = []

# For each row in a varible,
for row in customerlocations['coords']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
customerlocations['latitude'] = lat                            #SEPERATES LATS AND LONG INTO DIFF COLUMNS
customerlocations['longitude'] = lon

In [8]:
#create darkmap 
map_dark = folium.Map(location=[41.850033, -87.6500523], tiles = 'CartoDB dark_matter', zoom_start = 4)

In [9]:
tweetlocations = customerlocations[['latitude', 'longitude']]
locationlist = tweetlocations.values.tolist()
locationlistsize = len(locationlist)
locationlistsize

693

In [10]:
customerlocations.columns

Index(['cities', 'coords', 'latitude', 'longitude'], dtype='object')

In [11]:
customerlocations.insert(0, 'radiussize', 500)

In [12]:
for point in range(0, locationlistsize):
    folium.Circle(locationlist[point],
                  popup=customerlocations['cities'][point],
                  radius=str(customerlocations['radiussize'][point] * 100),
                  color='#17cbef',
                  fill=True,
                  opacity=0.8,
                  fill_color='#17cbef',
                  stroke=True,
                  weight=1.0).add_to(map_dark)
    
map_dark

In [13]:
from folium.plugins import HeatMap
#create heatmap
map_heat = folium.Map(location=[41.850033, -87.6500523], tiles = 'CartoDB dark_matter', zoom_start = 4)


In [14]:
for point in range(0, locationlistsize):
    folium.Circle(locationlist[point],
                  popup=customerlocations['cities'][point],
                  radius=str(customerlocations['radiussize'][point] * 100),
                  color='#17cbef',
                  fill=True,
                  opacity=0.8,
                  fill_color='#17cbef',
                  stroke=True,
                  weight=1.0).add_to(map_heat)
#convert to array format for our heatmap    
tweetarray = customerlocations[['latitude', 'longitude']].values

#plotheatmap
map_heat.add_child(HeatMap(tweetarray))
